# 第1题
如果你在相同训练集上训练 5 个不同的模型，它们都有 95% 的准确率，那么你是否可以通过组合这个模型来得到更好的结果？如果可以那怎么做呢？如果不可以请给出理由。

**
答：可以尝试将5个模型组合成一个voting ensemble。如果模型选择不同，比如有支持向量机，决策树，逻辑回归等，结果会比较好；而如果他们基于不同的实例训练（bagging，pasting），结果会更好
**

# 第2题
软投票和硬投票分类器之间有什么区别？

**
答：硬投票分类器只统计emsemble中每个分类器的结果，并挑选获得最多投票的数；软投票分类器计算分类器对每个类别的平均概率估计，然后以最高概率挑选类别，一般软投票表现更好，但必须分类器能估计概率时才可以用，对于SVM分类器，必须设置probability=True才可以使用软类别估计器
**

# 第3题
是否有可能通过分配多个服务器来加速 bagging 集成系统的训练？pasting 集成，boosting 集成，随机森林，或 stacking 集成怎么样？

**
答：通过分配多个服务器，可以加速 bagging集成 的训练，因为 bagging集成 中的每个预测器都独立于其它预测器，同样，pasting集成与随机森林也一样。 但是，boosting集成中的每个预测器都是基于之前的预测器构建的，因此训练必然是连续的，所以难以将训练分布到多个服务器训练。 对于stacking 集成，第一层中的预测器是相互独立的，可以在多台服务器上并行进行训练，不过，blender层预测器只能在第一层中的预测变量全部被训练后才能被训练。
**

# 第4题
out-of-bag 评估的好处是什么？

**
通过out-of-bag，可以使用每个预测器并未使用的那部分实例进行评估，这可以对ensemble进行公正的评估，而不需要额外的验证集，因此，你可以使用更多的实例进行训练，这样你的模型的性能会稍微好点
**

# 第5题
是什么使 Extra-Tree 比规则随机森林更随机呢？这个额外的随机有什么帮助呢？那这个 Extra-Tree 比规则随机森林谁更快呢？

* 对于随机森林，树在生长过程中，节点的分裂仅仅考虑特征的随机子集；
* 对于Extra-Tree，它不仅使用随机子集，并且，节点的分裂不使用常规决策树使用的最佳阈值分割，而是对每个特征设置随机阈值进行分割。
* 这种所处的随机过程更像一种正则化：如果随机森林发生过拟合，Extra-Tree可能表现会更好。在训练时，Extra-Tree更快，但是在预测时，两者差不多。


# 第6题
如果你的 Adaboost 模型欠拟合，那么你需要怎么调整超参数？

In [ ]:
sklearn.ensemble.AdaBoostClassifier(base_estimator=None, 
                                    n_estimators=50, 
                                    learning_rate=1.0, 
                                    algorithm='SAMME.R', 
                                    random_state=None)

* (1) 增加n_estimators
* (2) 降低base_estimator的正则化超参数
* (3) 稍微提高学习率

# 第7题
如果你的梯度提升过拟合，那么你应该调高还是调低学习率呢？

** 
答：应该尝试降低学习率，也可以使用提前停止找到最佳的预测器
**

# 第8题
导入 MNIST 数据（第三章中介绍），把它切分进一个训练集，一个验证集，和一个测试集（例如 40000 个实例进行训练，10000 个进行验证，10000 个进行测试）。然后训练多个分类器，例如一个随机森林分类器，一个 Extra-Tree 分类器和一个 SVM。接下来，尝试将它们组合成集成，使用软或硬投票分类器来胜过验证集上的所有集合。一旦找到了，就在测试集上实验。与单个分类器相比，它的性能有多好？

**（1）分割训练集，验证集，测试集**

In [ ]:
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split

mnist = fetch_mldata('MNIST origin')

# 分割训练集，验证集，测试集
X_train_val, X_test, y_train_val, y_test = train_test_split(mnist.data, mnist.target, test_size=10000, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=10000, random_state=42)

**（2）训练多个分类器**

In [ ]:
# 随机森林分类器
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

rfc_clf = RandomForestClassifier(random_state=42)
etc_clf = ExtraTreesClassifier(random_state=42)
svm_clf = LinearSVC(random_state=42)
mlp_clf = MLPClassifier(randoom_state=42)

estimators = [rfc_clf, etc_clf, svm_clf, mlp_clf]
for estimator in estimators:
    print('Train the: ', estimator)
    estimator.fit(X_train, y_train)
    
[estimator.score(X_val, y_val) for estimator in estimators]

**（3）硬分类集合**

In [ ]:
from sklearn.ensemble import VotingClassifier

named_estimators = [('rfc_clf', rfc_clf),
                    ('etc_clf', etc_clf),
                    ('svm_clf', svm_clf),
                    ('mlp_clf', mlp_clf)]

voting_clf = VotingClassifier(estimators=named_estimators,voting='hard')
voting_clf.fit(X_train, y_train)

voting_clf.score(X_val, y_val)

In [ ]:
# 移除SVM，查看性能变化
del voting_clf.estimators_[2]
voting_clf.score(X_val, y_val)

**（4）软分类集合**

In [ ]:
voting_clf.voting = 'soft'
voting_clf.score(X_val, y_val)

**（5）测试集性能评估**

In [ ]:
voting_clf.score(X_test, y_test)

In [ ]:
[estimator.score(X_test, y_test) for estimator in voting_clf.estimators_]

# 第9题
运行练习8中的所有单个分类器，对验证集进行预测，得到验证集的预测向量，将此向量作为X，依然将验证的y作为y，使用算法进行blender的训练，得到blender分类器。

然后，使用训练的blender分类器，对测试集进行预测，查看此分类器结果相比于第8题的投票分类器如何？

**（1）所有模型预测验证集得到的结果作为blender的输入向量，y_val作为输出向量**

In [ ]:
X_val_predictions = np.empty((len(X_val),len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)

X_val_predictions

**（2）使用RandomForestClassifier训练blender**

In [ ]:
# 可以尝试其他方法，不局限于随机森林
rfc_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rfc_blender.fit(X_val_predicts, y_val)
rfc_blender.oob_score_

**（3）使用训练的bleder对测试集进行预测**

In [ ]:
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)
    
y_test_pred = rfc_blender.predict(X_test_predictions)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)